In [20]:
import cv2
import math
import cvzone
import random
import numpy as np
import mediapipe as mp

# Initialize the mediapipe hands class.
mp_hands = mp.solutions.hands

# Set up the Hands functions for images and videos.
hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7)

# Initialize the mediapipe drawing class.
mp_drawing = mp.solutions.drawing_utils



font = cv2.FONT_HERSHEY_SIMPLEX
fontScale = 0.6
color = (255, 0, 0)
thickness = 2
INDEX_A = 8
INDEX_B = 6
MIDDLE_A = 12
MIDDLE_B = 10
RING_A = 16
RING_B = 14
PINKY_A = 20
PINKY_B = 18
THUMB_A = 4
THUMB_B = 2


class Snake:
    def __init__(self, pathFood):
        self.points = []
        self.distances = []
        self.currentLength = 0
        self.allowedLength = 150
        self.previousHead = 0, 0
        self.score = 0
        self.gameOver = False
        
        self.imgFood = cv2.imread(pathFood, cv2.IMREAD_UNCHANGED)
        self.imgFood = cv2.resize(self.imgFood, (90, 70))
        self.hFood, self.wFood, _ = self.imgFood.shape
        self.foodPoint = 0, 0
        self.randomFoodLocation()
        
        
    def randomFoodLocation(self):
        self.foodPoint = random.randint(50, 600), random.randint(50, 400)
    
    def update(self, imgMain, currentHead):
        if self.gameOver:
            cvzone.putTextRect(imgMain, "Game Over", 
                            [20, imgMain.shape[0]//2],
                            scale=4, thickness=3, offset=10)
            cvzone.putTextRect(imgMain, f"Your Score: {self.score}", 
                            [20, imgMain.shape[0]//2+100],
                            scale=4, thickness=3, offset=10)
        else:
            px, py = self.previousHead
            cx, cy = currentHead
            self.points.append([cx, cy])

            distance = math.hypot(cx - px, cy - py)
            self.distances.append(distance)
            self.currentLength += distance
            self.previousHead = cx, cy


            if self.currentLength > self.allowedLength:
                for i, distance in enumerate(self.distances):
                    self.currentLength -= distance
                    self.distances.pop(i)
                    self.points.pop(i)
                    if self.currentLength < self.allowedLength:
                        break

            rx, ry = self.foodPoint
            if (rx-self.wFood // 2 < cx < rx+self.wFood // 2) and (ry-self.hFood//2 < cy < ry+self.hFood//2):
                self.randomFoodLocation()
                self.allowedLength += 20
                self.score += 1

            if self.points:
                for i, point in enumerate(self.points):
                    if i != 0:
                        cv2.line(imgMain, self.points[i-1], self.points[i], (0, 0, 255), 20)
                cv2.circle(imgMain, self.points[-1], 20, (200, 0, 200), cv2.FILLED)

            pts = np.array(self.points[:-2], np.int32)
            pts = pts.reshape((-1, 1, 2))
            cv2.polylines(imgMain, [pts], False, (0, 200, 0), 3)
            minDist = cv2.pointPolygonTest(pts, (int(cx), int(cy)), True)
            if -1 < minDist <= 1:
                self.gameOver = True

                self.points = []
                self.distances = []
                self.currentLength = 0
                self.allowedLength = 150
                self.previousHead = 0, 0

                self.randomFoodLocation()


            try:
                imgMain = cvzone.overlayPNG(imgMain, self.imgFood, (rx-self.wFood//2, ry-self.hFood//2))
            except:
                pass
        return imgMain, self.score

In [21]:
stream = cv2.VideoCapture(0)
window_name = "AI Calculator"
snake = Snake("food.png")
# cv2.namedWindow(window_name, cv2.WINDOW_NORMAL);
# cv2.setWindowProperty(window_name, cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN);
# cv2.setWindowProperty(window_name, cv2.WND_PROP_TOPMOST, 1)

frameCX = 0
overlap = False
size = 20
size = 20
text_value = None
sent = ""
answer = ""

while True:
    ret, frame = stream.read()
    image = cv2.flip(frame, 1)
    img_h, img_w = image.shape[:2]
    if not ret:
        break
    
    imgRGB = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(imgRGB)
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            image = cv2.blur(image, (45, 45))
            mp_drawing.draw_landmarks(image, landmark_list = hand_landmarks,
                                      connections = mp_hands.HAND_CONNECTIONS,
                                      landmark_drawing_spec=mp_drawing.DrawingSpec(color=(255,255,255), thickness=2, circle_radius=2),
                                      connection_drawing_spec=mp_drawing.DrawingSpec(color=(0,255,0), thickness=2, circle_radius=2))
            points = np.array([np.multiply([p.x, p.y], [img_w, img_h]).astype("int") for p in hand_landmarks.landmark], np.int32)
        
        image = cv2.circle(image, points[INDEX_A], 10, (0, 0, 255), cv2.FILLED)
    
        image, score = snake.update(image, points[INDEX_A])
        cvzone.putTextRect(image, f"Score: {str(score)}", [50, 50],
                              scale=3, thickness=3, offset=10)
    
    cv2.imshow(window_name, image)
    key = cv2.waitKey(1)
    if key == ord('r'):
        snake.score = 0
        snake.gameOver = False
    if key == ord('q'):
        break
    
stream.release()
cv2.destroyAllWindows()

In [13]:
stream.release()
cv2.destroyAllWindows()